# **Proyecto I- Parte 3 (Core)**

**Análisis y Predicción de Ventas en una Tienda de Retail**

Parte III: Transformación y Análisis Avanzado de Datos con Pandas



## ***0) Preparativos***
- Instalación de pandas.
- Carga de datos.
- Revisar valores únicos y conteo de estos

In [68]:
import pandas as pd

df = pd.read_csv('/content/retail_sales_dataset.csv')
print(df,'\n')

for column in df.columns:
    print(f"\nConteo de valores únicos en la columna '{column}':")
    print(df[column].value_counts().sort_index())

     Transaction ID        Date Customer ID  Gender  Age Product Category  \
0                 1  2023-11-24     CUST001    Male   34           Beauty   
1                 2  2023-02-27     CUST002  Female   26         Clothing   
2                 3  2023-01-13     CUST003    Male   50      Electronics   
3                 4  2023-05-21     CUST004    Male   37         Clothing   
4                 5  2023-05-06     CUST005    Male   30           Beauty   
..              ...         ...         ...     ...  ...              ...   
995             996  2023-05-16     CUST996    Male   62         Clothing   
996             997  2023-11-17     CUST997    Male   52           Beauty   
997             998  2023-10-29     CUST998  Female   23           Beauty   
998             999  2023-12-05     CUST999  Female   36      Electronics   
999            1000  2023-04-12    CUST1000    Male   47      Electronics   

     Quantity  Price per Unit  Total Amount  
0           3              50

#**1) Transformación de Datos**
- Crea nuevas columnas: Basándonos en los datos existentes, crea nuevas columnas que sean útiles para el análisis. Por ejemplo, calcula el ingreso total por venta y normaliza las ventas.
- Clasifica los datos: Crea una columna que clasifique las ventas en categorías significativas (e.g., ‘Alta’, ‘Media’, ‘Baja’).



In [69]:
# Como las ventas totales ya están calculadas, borraré la columna
df = df.drop('Total Amount', axis=1)

# Ahora creo la nueva columna de ventas
df['Total Sales'] = df['Quantity'] * df['Price per Unit']

# Normalizar la columna 'Total Sales'
max_value = df['Total Sales'].max()
min_value = df['Total Sales'].min()
df['n_Total Sales'] = df['Total Sales'].apply(lambda x: (x - min_value) / (max_value - min_value))
print(df)

# Se verificaron algunos datos y se puede afirmar que los calculos quedaron bien

     Transaction ID        Date Customer ID  Gender  Age Product Category  \
0                 1  2023-11-24     CUST001    Male   34           Beauty   
1                 2  2023-02-27     CUST002  Female   26         Clothing   
2                 3  2023-01-13     CUST003    Male   50      Electronics   
3                 4  2023-05-21     CUST004    Male   37         Clothing   
4                 5  2023-05-06     CUST005    Male   30           Beauty   
..              ...         ...         ...     ...  ...              ...   
995             996  2023-05-16     CUST996    Male   62         Clothing   
996             997  2023-11-17     CUST997    Male   52           Beauty   
997             998  2023-10-29     CUST998  Female   23           Beauty   
998             999  2023-12-05     CUST999  Female   36      Electronics   
999            1000  2023-04-12    CUST1000    Male   47      Electronics   

     Quantity  Price per Unit  Total Sales  n_Total Sales  
0           3  

In [70]:
# Clasificando las ventas en categorías
df['Sales Classification'] = df['Total Sales'].apply(lambda x: 'Low' if x < 500 else ('Medium' if 500 <= x <= 1000 else 'High'))
print(df)

     Transaction ID        Date Customer ID  Gender  Age Product Category  \
0                 1  2023-11-24     CUST001    Male   34           Beauty   
1                 2  2023-02-27     CUST002  Female   26         Clothing   
2                 3  2023-01-13     CUST003    Male   50      Electronics   
3                 4  2023-05-21     CUST004    Male   37         Clothing   
4                 5  2023-05-06     CUST005    Male   30           Beauty   
..              ...         ...         ...     ...  ...              ...   
995             996  2023-05-16     CUST996    Male   62         Clothing   
996             997  2023-11-17     CUST997    Male   52           Beauty   
997             998  2023-10-29     CUST998  Female   23           Beauty   
998             999  2023-12-05     CUST999  Female   36      Electronics   
999            1000  2023-04-12    CUST1000    Male   47      Electronics   

     Quantity  Price per Unit  Total Sales  n_Total Sales Sales Classificat

## *Análisis*
1. Si bien se realizó un paso "innecesario" al eliminar y crear nuevamente dicha columna, en la siguiente sección se agregan nuevas columnas para poder hacer una agrupación puntual.
2. La categorización de las ventas se realizó con tres niveles, bajo: 0 a <500, medio: entre 500 y 1.000, alto: >1.000


# **2) Agrupación y Agregación**
- Agrupación por múltiples columnas: Realiza agrupaciones por categorías como producto y tienda, producto y mes, etc.
-Aplicar funciones de agregación: Utiliza funciones como sum, mean, count, min, max, std, y var para obtener estadísticas descriptivas de cada grupo.

In [71]:
# Para poder sacar el mes, lo mejor es verificar que la columna Date sea fecha
print(df.dtypes)

# Cómo es tipo object, toca hacer el cambio en el tipo de dato para poder hacerlo más fácil con métodos de Pandas
df['Date'] = pd.to_datetime(df['Date'])

# Extraer el número del mes
df['Month Number'] = df['Date'].dt.month

# Con otra librería se podría crear la nueva columna con el nombre del mes a través del número
# Esto también se podría hacer con una función o una lista o un array o un bucle
import calendar

# Obtener el nombre del mes en inglés
df['Month Name'] = df['Month Number'].apply(lambda x: calendar.month_name[x])
print('\n',df.dtypes)
print('\n',df)

Transaction ID            int64
Date                     object
Customer ID              object
Gender                   object
Age                       int64
Product Category         object
Quantity                  int64
Price per Unit            int64
Total Sales               int64
n_Total Sales           float64
Sales Classification     object
dtype: object

 Transaction ID                   int64
Date                    datetime64[ns]
Customer ID                     object
Gender                          object
Age                              int64
Product Category                object
Quantity                         int64
Price per Unit                   int64
Total Sales                      int64
n_Total Sales                  float64
Sales Classification            object
Month Number                     int32
Month Name                      object
dtype: object

      Transaction ID       Date Customer ID  Gender  Age Product Category  \
0                 1 2023-11-24   

In [72]:
# Agrupar por Categoría de producto y mes (el número para organizar de 1 a 12)
grouped_1 = df.groupby(['Product Category','Month Number','Month Name'])

# Suma y promedio de ventas por producto y mes
sales_Product_Category = grouped_1['Total Sales'].agg(['count','sum', 'min', 'mean', 'max', 'std','var']).round(1)

sales_Product_Category

count    sum  min   mean   max  \
Product Category Month Number Month Name                                   
Beauty           1            January        26  13930   25  535.8  2000   
                 2            February       26  14035   25  539.8  2000   
                 3            March          21  10545   25  502.1  1500   
                 4            April          29  11905   25  410.5  2000   
                 5            May            28  12450   30  444.6  2000   
                 6            June           25  10995   25  439.8  2000   
                 7            July           27  16090   30  595.9  2000   
                 8            August         24   9790   25  407.9  1200   
                 9            September      20   6320   25  316.0  2000   
                 10           October        31  15355   25  495.3  2000   
                 11           November       25   9700   25  388.0  2000   
                 12           December       25  12400   25  496.0  2000   
Clothing         1            January        26  13125   25  504.8  2000   
                 2            February       33  14560   25  441.2  2000   
                 3            March          38  15065   25  396.4  2000   
                 4            April          36  13940   25  387.2  2000   
                 5            May            37  17455   25  471.8  2000   
                 6            June           28  10170   25  363.2  1500   
                 7            July           19   8250   25  434.2  2000   
                 8            August         32  12455   25  389.2  2000   
                 9            September      20   9975   50  498.8  2000   
                 10           October        30  13315   25  443.8  2000   
                 11           November       26  15200   25  584.6  2000   
                 12           December       26  12070   30  464.2  2000   
Electronics      1            January        26   9925   25  381.7  2000   
                 2            February       26  15465   30  594.8  2000   
                 3            March          14   3380   30  241.4  1000   
                 4            April          21   8025   25  382.1  1500   
                 5            May            40  23245   25  581.1  2000   
                 6            June           24  15550   30  647.9  2000   
                 7            July           26  11125   30  427.9  2000   
                 8            August         38  14715   25  387.2  2000   
                 9            September      25   7325   30  293.0  2000   
                 10           October        35  17910   25  511.7  2000   
                 11           November       27  10020   25  371.1  1500   
                 12           December       40  20220   25  505.5  2000   

                                            std       var  
Product Category Month Number Month Name                   
Beauty           1            January     658.8  434041.4  
                 2            February    697.8  486861.0  
                 3            March       482.9  233206.4  
                 4            April       519.7  270036.3  
                 5            May         571.6  326675.8  
                 6            June        512.2  262396.8  
                 7            July        626.1  391955.8  
                 8            August      451.7  204060.7  
                 9            September   506.0  255998.9  
                 10           October     552.6  305339.9  
                 11           November    583.1  340039.6  
                 12           December    583.3  340281.3  
Clothing         1            January     540.9  292543.0  
                 2            February    551.7  304396.9  
                 3            March       497.6  247568.8  
                 4            April       576.2  331987.8  
                 5            May         567.6  322222.5  
             

In [73]:
# Agrupar por Categoría de producto y mes (el número para organizar de 1 a 12)
grouped_2 = df.groupby(['Gender','Month Number','Month Name'])

# Suma y promedio de ventas por producto y mes
sales_Gender = grouped_2['Total Sales'].agg(['count','sum', 'min', 'mean', 'max', 'std','var']).round(1)

sales_Gender

count    sum  min   mean   max    std  \
Gender Month Number Month Name                                          
Female 1            January        40  24725   25  618.1  2000  695.1   
       2            February       36  14395   25  399.9  1500  500.0   
       3            March          41  13320   25  324.9  1500  409.1   
       4            April          50  17300   25  346.0  1500  455.8   
       5            May            45  23930   25  531.8  2000  636.4   
       6            June           39  16875   25  432.7  2000  499.8   
       7            July           37  16885   30  456.4  2000  552.3   
       8            August         50  22080   25  441.6  2000  568.3   
       9            September      35  16535   30  472.4  2000  592.3   
       10           October        43  26600   25  618.6  2000  624.3   
       11           November       39  14890   25  381.8  2000  554.0   
       12           December       55  25305   25  460.1  2000  504.1   
Male   1            January        38  12255   25  322.5  1500  404.5   
       2            February       49  29665   25  605.4  2000  682.8   
       3            March          32  15670   25  489.7  2000  514.8   
       4            April          36  16570   25  460.3  2000  632.2   
       5            May            60  29220   25  487.0  2000  576.3   
       6            June           38  19840   25  522.1  2000  601.8   
       7            July           35  18580   25  530.9  2000  607.4   
       8            August         44  14880   25  338.2  1500  434.3   
       9            September      30   7085   25  236.2  2000  402.6   
       10           October        53  19980   25  377.0  2000  579.6   
       11           November       39  20030   25  513.6  1500  551.2   
       12           December       36  19385   25  538.5  2000  630.2   

                                     var  
Gender Month Number Month Name            
Female 1            January     483132.9  
       2            February    249977.8  
       3            March       167365.6  
       4            April       207711.2  
       5            May         404969.5  
       6            June        249816.9  
       7            July        305006.5  
       8            August      322912.7  
       9            September   350805.0  
       10           October     389805.1  
       11           November    306936.2  
       12           December    254096.8  
Male   1            January     163610.5  
       2            February    466236.3  
       3            March       264991.8  
       4            April       399692.8  
       5            May         332079.8  
       6            June        362169.8  
       7            July        368962.5  
       8            August      188598.9  
       9            September   162090.8  
       10           October     335984.9  
       11           November    303849.9  
       12           December    397095.5

## *Análisis*
1. Se generaron columnas nuevas para número de mes y nombre de mes, y con ellas se agrupo por categoría de producto y mes
2. Se agregaron funciones básicas para consultar las ventas en las dos agrupaciones generadas, una por género y meses con las ventas, y otra por categoría de producto y meses con las ventas
3. En cuanto a las categorías de producto, no pareciera haber mucha diferencia, incluso son ventas "parejas" o similares entre los grupos, si bien mensualmente hay alguna categoría que sobresale, esta misma tiene las menores ventas en otro mes, así que todas se compensan entre sí.
4. Parece que hay epocas del año donde los hombres compran más que las mujeres, y viceversa, esto puede servir para analizar patrones de ventas, y poder también relacionarlo con lo que compran, pero al hacer la consulta más extensa, ya no me da información completa, sino que me muestra el resultado "mocho"
Me refiero a cuando se agrupa así:          
        -->grouped_2 = df.groupby(['Gender','Product Category','Month Number','Month Name'])

# **3) Análisis Personalizado con apply**

- Función personalizada: Aplica funciones personalizadas para realizar análisis específicos que no se pueden lograr con las funciones de agregación estándar.
- Ejemplo de uso avanzado: Calcula la desviación de cada venta respecto a la media de su grupo.

In [74]:
# ESTO ERA UN ENSAYO PARA REVISAR SI JUSTIFICABA INCLUIRLO EN EL DataFrame ORIGINAL
# Mes con venta más alta
def mes_maximo_ventas(grupo):
    max_venta_mes = grupo.loc[grupo['Total Sales'].idxmax(), 'Month Name']
    grupo['Max_Sales_Month'] = max_venta_mes
    return grupo

grouped_1_1 = df.groupby(['Product Category']).apply(mes_maximo_ventas)

grouped_1_1

<ipython-input-74-acf7c2bb582c>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_1_1 = df.groupby(['Product Category']).apply(mes_maximo_ventas)


Transaction ID       Date Customer ID  Gender  Age  \
Product Category                                                           
Beauty           0                 1 2023-11-24     CUST001    Male   34   
                 4                 5 2023-05-06     CUST005    Male   30   
                 5                 6 2023-04-25     CUST006  Female   45   
                 11               12 2023-10-30     CUST012    Male   35   
                 20               21 2023-01-14     CUST021  Female   50   
...                              ...        ...         ...     ...  ...   
Electronics      988             989 2023-12-28     CUST989  Female   44   
                 991             992 2023-08-21     CUST992  Female   57   
                 992             993 2023-02-06     CUST993  Female   48   
                 998             999 2023-12-05     CUST999  Female   36   
                 999            1000 2023-04-12    CUST1000    Male   47   

                     Product Category  Quantity  Price per Unit  Total Sales  \
Product Category                                                               
Beauty           0             Beauty         3              50          150   
                 4             Beauty         2              50          100   
                 5             Beauty         1              30           30   
                 11            Beauty         3              25           75   
                 20            Beauty         1             500          500   
...                               ...       ...             ...          ...   
Electronics      988      Electronics         1              25           25   
                 991      Electronics         2              30           60   
                 992      Electronics         3              50          150   
                 998      Electronics         3              50          150   
                 999      Electronics         4              30          120   

                      n_Total Sales Sales Classification  Month Number  \
Product Category                                                         
Beauty           0         0.063291                  Low            11   
                 4         0.037975                  Low             5   
                 5         0.002532                  Low             4   
                 11        0.025316                  Low            10   
                 20        0.240506               Medium             1   
...                             ...                  ...           ...   
Electronics      988       0.000000                  Low            12   
                 991       0.017722                  Low             8   
                 992       0.063291                  Low             2   
                 998       0.063291                  Low            12   
                 999       0.048101                  Low             4   

                     Month Name Max_Sales_Month  
Product Category                                 
Beauty           0     November        November  
                 4          May        November  
                 5        April        November  
                 11     October        November  
                 20     January        November  
...                         ...             ...  
Electronics      988   December         January  
                 991     August         January  
                 992   February         January  
                 998   December         January  
                 999      April         January  

[1000 rows x 14 columns]

In [78]:
# Función perdonalizada: Identificar ventas por encima de la media del grupo
df['Above_Average'] = df.groupby(['Product Category', 'Month Number'])['Total Sales'].transform(
    lambda x: x > x.mean()
)

In [79]:
# Esta parte solicité ayuda a ChatGPT porque no entendía bien qué era loq ue debía hacer, aunque él fue el que me sugirió hacer una función en vez de un apply directamente, ya que los que había pensado me daban error
def desviacion_respecto_media(grupo):
    grupo['Deviation_from_Mean'] = grupo['Total Sales'] - grupo['Total Sales'].mean().round(1)
    return grupo

# Aplicar la función personalizada a los grupos
resultado = df.groupby(['Product Category', 'Month Number']).apply(desviacion_respecto_media)

# Mostrar resultado
resultado

<ipython-input-79-3f8617484b08>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  resultado = df.groupby(['Product Category', 'Month Number']).apply(desviacion_respecto_media)


Transaction ID       Date Customer ID  \
Product Category Month Number                                              
Beauty           1            20               21 2023-01-14     CUST021   
                              31               32 2023-01-04     CUST032   
                              188             189 2023-01-30     CUST189   
                              210             211 2024-01-01     CUST211   
                              224             225 2023-01-11     CUST225   
...                                           ...        ...         ...   
Electronics      12           915             916 2023-12-24     CUST916   
                              938             939 2023-12-18     CUST939   
                              970             971 2023-12-05     CUST971   
                              988             989 2023-12-28     CUST989   
                              998             999 2023-12-05     CUST999   

                                   Gender  Age Product Category  Quantity  \
Product Category Month Number                                               
Beauty           1            20   Female   50           Beauty         1   
                              31     Male   30           Beauty         3   
                              188    Male   63           Beauty         1   
                              210    Male   42           Beauty         3   
                              224  Female   57           Beauty         4   
...                                   ...  ...              ...       ...   
Electronics      12           915  Female   32      Electronics         1   
                              938  Female   46      Electronics         1   
                              970  Female   27      Electronics         4   
                              988  Female   44      Electronics         1   
                              998  Female   36      Electronics         3   

                                   Price per Unit  Total Sales  n_Total Sales  \
Product Category Month Number                                                   
Beauty           1            20              500          500       0.240506   
                              31               30           90       0.032911   
                              188              50           50       0.012658   
                              210             500         1500       0.746835   
                              224              25          100       0.037975   
...                                           ...          ...            ...   
Electronics      12           915              50           50       0.012658   
                              938             300          300       0.139241   
                              970              50          200       0.088608   
                              988              25           25       0.000000   
                              998              50          150       0.063291   

                                  Sales Classification  Month Number  \
Product Category Month Number                                          
Beauty           1            20                Medium             1   
                              31                   Low             1   
                              188                  Low             1   
                              210                 High             1   
                              224                  Low             1   
...                                                ...           ...   
Electronics      12           915                  Low            12   
                              938                  Low            12   
                              970                  Low            12   
                              988                  Low            12   
                              998                  Low            12   

                                  Month Name  Above_A

## *Análisis*
1. Si bien el primer código de este bloque me devolvía el mes con mas ventas por categoría (grouped_1_1), dicha información puede no ser de utilidad dependiendo del análisis.
2. Al realizar el segundo código, para revisar si el valor de la venta está por encima de la media del grupo, y que me devuelva un booleano, parece ser más adecuado para mirar los patrones de meses donde hay más ventas en cada categoría.
3. El tercer código que me calcula la desviación de la media, me confirma el punto anterior, ya que, donde la venta de la categoría fue por encima del promedio, me devuelve por un lado TRUE y aquí me da un valor positivo


PD: Creo que algo esta mal en la agrupación, y que no encontré como corregir.
Porque si bien quiero que me de los resultados en la agrupación, me lo termina dando en todo el df, así que me carga todo el df con la agrupación dada, y no es lo que quiero como tal

# **4) Documentación**
- Comentarios claros: Documenta claramente cada paso del análisis, explicando qué se hizo y por qué se hizo.
- Código legible: Asegúrate de que el código sea legible y esté bien comentado.